# Semantic search Movies

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import json
from fastapi import FastAPI

: 

# Instance the sentence transformer embedding model

In [2]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
encoder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Loading the data

In [4]:
with open("sample_mflix.embedded_movies.json", "r") as f:
    data = json.load(f)

# Start the qdrant client

In [9]:
client = QdrantClient(url="http://localhost:6333")

# Create the "movie" collection

In [10]:
client.create_collection(
    collection_name="movies",
    vectors_config=models.VectorParams(size=encoder.get_sentence_embedding_dimension(),
    distance= models.Distance.COSINE
    ),
)

True

# Upload the json data to the collection as points

In [16]:
client.upload_points(
    collection_name="movies",
    points=[
        models.PointStruct(
            id=idx, 
            vector=encoder.encode(doc['fullplot']).tolist(), 
            payload=doc
        ) 
        for idx, doc in enumerate(data) 
        if doc.get('fullplot') is not None
    ]
)

# Search for similar movies to query "mars"

In [21]:
searches = client.query_points(
    collection_name="movies",
    query = encoder.encode("mars").tolist(),
    limit=3
).points

In [22]:
searches

[ScoredPoint(id=926, version=13, score=0.5237663, payload={'fullplot': "In the near future, Earth is dying. A new colony on Mars could be humanity's only hope. A team of American astronauts, each a specialist in a different field, is making the first manned expedition to the red planet and must struggle to overcome the differences in their personalities, backgrounds and ideologies for the overall good of the mission. When their equipment suffers life-threatening damage and the crew must depend on one another for survival on the hostile surface of Mars, their doubts, fears and questions about God, man's destiny and the nature of the universe become defining elements in their fates. In this alien environment they must come face to face with their most human selves.", 'imdb': {'rating': 5.6, 'votes': 45225, 'id': 199753}, 'plot': 'Astronauts, and their robotic dog AMME, search for solutions to save a dying Earth by searching on Mars, only to have the mission go terribly awry.', 'genres': 

In [23]:
for search in searches:
    print(search.payload["title"])
    print(search.payload["fullplot"])
    print("----")

Red Planet
In the near future, Earth is dying. A new colony on Mars could be humanity's only hope. A team of American astronauts, each a specialist in a different field, is making the first manned expedition to the red planet and must struggle to overcome the differences in their personalities, backgrounds and ideologies for the overall good of the mission. When their equipment suffers life-threatening damage and the crew must depend on one another for survival on the hostile surface of Mars, their doubts, fears and questions about God, man's destiny and the nature of the universe become defining elements in their fates. In this alien environment they must come face to face with their most human selves.
----
Ghosts of Mars
200 years in the future a Martian police unit is dispatched to transport a dangerous prisoner from a mining outpost back to justice. But when the team arrives they find the town deserted and some of the inhabitants possessed by the former inhabitants of the planet.
-